In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [2]:
loan_df = pd.read_csv("application_train.csv")
loan_df.drop_duplicates()
loan_df.dropna()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
71,100083,0,Cash loans,M,Y,Y,0,103500.0,573628.5,24435.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
124,100145,0,Cash loans,F,Y,Y,1,202500.0,260725.5,16789.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
152,100179,0,Cash loans,F,Y,N,0,202500.0,675000.0,53329.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
161,100190,0,Cash loans,M,Y,N,0,162000.0,263686.5,24781.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
255,100295,1,Cash loans,M,Y,N,1,225000.0,1019205.0,31032.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307358,456083,0,Cash loans,F,Y,Y,2,112500.0,361462.5,16051.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
307359,456084,0,Cash loans,F,Y,Y,1,99000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307407,456140,1,Cash loans,F,Y,Y,1,261000.0,711454.5,47673.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
307456,456195,0,Cash loans,F,Y,Y,0,94500.0,270000.0,15075.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0


In [3]:
loan_df.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

In [5]:
loan_df.select_dtypes(include=['object']).shape

(307511, 16)

In [7]:
target_column = 'TARGET'

# Set the correlation threshold
correlation_threshold = 0.1

# Compute the correlation matrix
correlation_matrix = loan_df.select_dtypes(exclude=['object']).corr()

# Get the absolute correlation values with the target column
correlation_with_target = correlation_matrix[target_column].abs()

# Filter the columns based on the threshold
columns_to_drop = correlation_with_target[correlation_with_target < correlation_threshold].index

# Drop the columns from the dataframe
df_filtered = loan_df.drop(columns=columns_to_drop)

# Print the filtered dataframe
df_filtered

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,0.083037,0.262949,0.139376,reg oper account,block of flats,"Stone, brick",No
1,0,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,0.311267,0.622246,NaN,reg oper account,block of flats,Block,No
2,0,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN
3,0,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,0.650442,NaN,NaN,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,0.322738,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services,0.145570,0.681632,NaN,reg oper account,block of flats,"Stone, brick",No
307507,0,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,MONDAY,XNA,NaN,0.115992,NaN,reg oper account,block of flats,"Stone, brick",No
307508,0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School,0.744026,0.535722,0.218859,reg oper account,block of flats,Panel,No
307509,1,Cash loans,F,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 1,NaN,0.514163,0.661024,NaN,block of flats,"Stone, brick",No


In [8]:
#check the types
df_filtered.dtypes

TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
OCCUPATION_TYPE                object
WEEKDAY_APPR_PROCESS_START     object
ORGANIZATION_TYPE              object
EXT_SOURCE_1                  float64
EXT_SOURCE_2                  float64
EXT_SOURCE_3                  float64
FONDKAPREMONT_MODE             object
HOUSETYPE_MODE                 object
WALLSMATERIAL_MODE             object
EMERGENCYSTATE_MODE            object
dtype: object

In [9]:
#review the data with the object
df_filtered.select_dtypes(include=['object'])

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services,reg oper account,block of flats,"Stone, brick",No
307507,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,MONDAY,XNA,reg oper account,block of flats,"Stone, brick",No
307508,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School,reg oper account,block of flats,Panel,No
307509,Cash loans,F,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 1,NaN,block of flats,"Stone, brick",No


In [10]:
#create dummies for the objects in the data
dummies_for_objects = pd.get_dummies(df_filtered.select_dtypes(include=['object']))

dummies_for_objects

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,True,False,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False
1,True,False,True,False,False,True,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
2,False,True,False,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,True,False,False,True,False,True,False,True,False,False,...,False,False,False,False,False,False,True,False,True,False
307507,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False
307508,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
307509,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False


In [11]:
#select columns of float type excluding any columns of type 'object', and then drop the 'TARGET' column
x_float = df_filtered.select_dtypes(exclude=['object']).drop(columns='TARGET')

#fills any missing values in x_float with 0
x_float = x_float.fillna(0)

x_float

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,0.083037,0.262949,0.139376
1,0.311267,0.622246,0.000000
2,0.000000,0.555912,0.729567
3,0.000000,0.650442,0.000000
4,0.000000,0.322738,0.000000
...,...,...,...
307506,0.145570,0.681632,0.000000
307507,0.000000,0.115992,0.000000
307508,0.744026,0.535722,0.218859
307509,0.000000,0.514163,0.661024


In [12]:
y= df_filtered.select_dtypes(exclude=['object'])['TARGET']
y

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

In [13]:
X = pd.concat([dummies_for_objects, x_float], axis=1)
X

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,True,False,False,True,False,True,False,False,True,False,...,False,False,False,True,False,True,False,0.083037,0.262949,0.139376
1,True,False,True,False,False,True,False,True,False,False,...,False,False,False,False,False,True,False,0.311267,0.622246,0.000000
2,False,True,False,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,0.000000,0.555912,0.729567
3,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,0.000000,0.650442,0.000000
4,True,False,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,0.000000,0.322738,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,True,False,False,True,False,True,False,True,False,False,...,False,False,False,True,False,True,False,0.145570,0.681632,0.000000
307507,True,False,True,False,False,True,False,False,True,False,...,False,False,False,True,False,True,False,0.000000,0.115992,0.000000
307508,True,False,True,False,False,True,False,False,True,False,...,False,False,True,False,False,True,False,0.744026,0.535722,0.218859
307509,True,False,True,False,False,True,False,False,True,False,...,False,False,False,True,False,True,False,0.000000,0.514163,0.661024


In [32]:
columns_with_nan = X.columns[X.isnull().any()]

columns_with_nan

Index([], dtype='object')

In [33]:
# Check the balance of our target values
y.value_counts()

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [34]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify= y)

In [35]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lreg_model = LogisticRegression(solver='lbfgs', random_state=1)


# Fit the model using training data
lreg_model.fit(x_train, y_train)

C:\Users\User\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [36]:
# Make a prediction using the testing data
test_data_predictions = lreg_model.predict(x_test)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
# Print the accuracy score of the model
accuracy_score(y_test,test_data_predictions)

0.9192616873487864

In [39]:
# Generate a confusion matrix for the model
con_matrix = confusion_matrix(y_test, test_data_predictions)
con_matrix_df = pd.DataFrame(
    con_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


con_matrix_df.head()

,Predicted 0,Predicted 1
Actual 0,70670,2
Actual 1,6205,1


In [40]:
# Print the classification report for the model
target_names = ["payment difficulty", "other cases"]
print(classification_report(y_test, test_data_predictions, target_names=target_names))

                    precision    recall  f1-score   support

payment difficulty       0.92      1.00      0.96     70672
       other cases       0.33      0.00      0.00      6206

          accuracy                           0.92     76878
         macro avg       0.63      0.50      0.48     76878
      weighted avg       0.87      0.92      0.88     76878



# Predict a Logistic Regression Model with Resampled Training Data

In [26]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
rand_oversampler = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
x_resampled, y_resampled = rand_oversampler.fit_resample(x_train, y_train)

In [27]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

TARGET
0    212014
1    212014
Name: count, dtype: int64

In [55]:
#x_resampled.value_counts()

# Use the LogisticRegression classifier and the resampled data to fit the model and make predictions

In [28]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_reg_model_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
log_reg_model_ros.fit(x_resampled, y_resampled)

# Make a prediction using the testing data
ros_predictions = log_reg_model_ros.predict(x_test)
pd.DataFrame({"Prediction": ros_predictions, "Actual": y_test}).reset_index(drop=True).head()

C:\Users\User\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Prediction,Actual
0,0,0
1,1,0
2,0,0
3,1,0
4,0,1


In [29]:
# Print the accuracy score of the model 
accuracy_score(y_test, ros_predictions)

0.6670438877182029

In [30]:
# Generate a confusion matrix for the model
ros_c_matrix = confusion_matrix(y_test, ros_predictions)
ros_c_matrix_df = pd.DataFrame(
    ros_c_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


ros_c_matrix_df.head()

,Predicted 0,Predicted 1
Actual 0,47249,23423
Actual 1,2174,4032


In [31]:
# Print the classification report for the model
target_names = ["payment difficulty", "other cases"]
print(classification_report(y_test, ros_predictions, target_names=target_names))

                    precision    recall  f1-score   support

payment difficulty       0.96      0.67      0.79     70672
       other cases       0.15      0.65      0.24      6206

          accuracy                           0.67     76878
         macro avg       0.55      0.66      0.51     76878
      weighted avg       0.89      0.67      0.74     76878

